In [7]:
import numpy as np

from camera import CameraMock

# Instantiate the camera.
camera = CameraMock()

# Generate 1200 images of good and defective images with and without low lighting.
data, labels = [], []
for low_lighting in [True, False]:
    for defect in [True, False]:
        for _ in range(300):
            img = camera.capture(has_defect=defect, low_lighting=low_lighting)
            data.append(np.array(img))
            labels.append(int(defect))

# Convert to numpy arrays.
data = np.array(data)
labels = np.array(labels)

# Reshape data to include the channel dimension
data = data.reshape((-1, 100, 100, 1))

In [8]:

from tensorflow.keras import layers, models

def create_cnn_model(input_shape):
    """Creates a CNN model for the given input shape"""
    # Init model.
    model = models.Sequential()

    # First Convolutional Layer.
    model.add(layers.Conv2D(16, (3, 3), activation="relu", input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))

    # Second Convolutional Layer.
    model.add(layers.Conv2D(32, (3, 3), activation="relu"))
    model.add(layers.MaxPooling2D((2, 2)))

    # Third Convolutional Layer.
    model.add(layers.Conv2D(32, (3, 3), activation="relu"))
    model.add(layers.MaxPooling2D((2, 2)))

    # Flatten the output and feed into a dense layer.
    model.add(layers.Flatten())
    model.add(layers.Dense(32, activation="relu"))
    model.add(layers.Dense(2, activation="softmax"))

    return model

In [9]:
from sklearn.model_selection import train_test_split

# Create and compile the model.
model = create_cnn_model(input_shape=(100, 100, 1))
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Split the data into training and testing datasets.
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=24)

# Normalize pixel values to be between 0 and 1.
X_train = X_train / 255.0
X_test = X_test / 255.0

# Train the model.
model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))

Epoch 1/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.5612 - loss: 0.6908 - val_accuracy: 0.5958 - val_loss: 0.6695
Epoch 2/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7001 - loss: 0.6221 - val_accuracy: 0.7667 - val_loss: 0.6165
Epoch 3/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.8113 - loss: 0.4185 - val_accuracy: 0.9875 - val_loss: 0.1334
Epoch 4/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9760 - loss: 0.1056 - val_accuracy: 1.0000 - val_loss: 0.0304
Epoch 5/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.9916 - loss: 0.0288 - val_accuracy: 1.0000 - val_loss: 0.0104
Epoch 6/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9984 - loss: 0.0090 - val_accuracy: 1.0000 - val_loss: 0.0057
Epoch 7/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 1.0000 - loss: 0.0057 - val_accuracy: 1.0000 - val_loss: 0.0040
Epoch 8/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 1.0000 - loss: 0.0032 - val_accuracy: 1.0000 - v

In [13]:
# Test model
for low_lighting in [True, False]:
    for defect in [True, False]:
        image = camera.capture(has_defect=defect, low_lighting=low_lighting)
        image_data = np.array(image).reshape((1, 100, 100, 1)) / 255.0  # Normalize and reshape

        results = model.predict(image_data, verbose=0)
        result = np.argmax(results[0])

        print(f"Defect: {defect}, Low Lighting: {low_lighting}")
        print(bool(result))

Defect: True, Low Lighting: True
True
Defect: False, Low Lighting: True
False
Defect: True, Low Lighting: False
True
Defect: False, Low Lighting: False
False


In [11]:
model.save("cnn_model.keras")